AC optimal power flow (OPF) problem for a simple power grid:

**Sets:**
- \($ \mathcal{B} $\): Set of buses (nodes) in the power grid.
- \( $\mathcal{L} $\): Set of transmission lines in the power grid.

**Parameters:**
- \($ P_{\text{min}, i} $\): Minimum power generation limit at bus \( $i$ \).
- \($ P_{\text{max}, i}$ \): Maximum power generation limit at bus \( $i $\).
- \( $C_i $\): Cost of generation at bus \( $i$ \).
- \( $P_{\text{d}, i}$ \): Power demand at bus \( $i$ \).
- \( $P_{\text{line}, (i,j)} $\): Maximum power flow limit on line \( $(i,j)$ \).
- \( $R_{(i,j)}$ \): Resistance of line \( $(i,j) $\).
- \( $X_{(i,j)}$ \): Reactance of line \( $(i,j)$ \).

**Variables:**
- \($ P_{\text{g}, i}$ \): Power generation at bus \( $i$ \).
- \( $V_i $\): Voltage magnitude at bus \( $i$ \).
- \( $\theta_i $\): Voltage phase angle at bus \( $i$ \).

**Objective Function:**
$$
\min \sum_{i \in \mathcal{B}} C_i \cdot P_{\text{g}, i}
$$

**Constraint:**

1. **Power Balance:** For each bus \( $i$ \):
$$
P_{\text{g}, i} - P_{\text{d}, i} = \sum_{(i,j) \in \mathcal{L}} \left( \frac{{V_i^2}}{{V_i V_j}} (R_{(i,j)} \cdot \cos(\theta_i - \theta_j) + X_{(i,j)} \cdot \sin(\theta_i - \theta_j)) \right) - \sum_{(j,i) \in \mathcal{L}} \left( \frac{{V_j^2}}{{V_i V_j}} (R_{(i,j)} \cdot \cos(\theta_i - \theta_j) + X_{(i,j)} \cdot \sin(\theta_i - \theta_j)) \right)
$$

2. **Line Flow Limits:** For each line \( $(i,j)$ \):
$$
V_i (R_{(i,j)} \cdot \cos(\theta_i - \theta_j) + X_{(i,j)} \cdot \sin(\theta_i - \theta_j)) \leq P_{\text{line}, (i,j)}
$$

3. **Voltage Magnitude Limits:** For each bus \( $i$ \):
$$
P_{\text{min}, i} \leq P_{\text{g}, i} \leq P_{\text{max}, i} \\
V_{\text{min}} \leq V_i \leq V_{\text{max}}
$$

In this formulation:

- The objective is to minimize the total generation cost, which is the sum of the generation cost at each bus multiplied by the corresponding power generation.
- The power balance constraint ensures that the power generated at each bus meets the local demand and accounts for the power flow through transmission lines.
- The line flow limits constrain the power flow on each transmission line to be within its maximum limit.
- The voltage magnitude limits restrict the voltage magnitude at each bus to lie within a specified range.

This formulation captures the essential aspects of the AC optimal power flow problem for a simple power grid. Adjustments and extensions can be made to accommodate additional features or complexities in the power system.

In [9]:
from pyomo.environ import *

In [11]:
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Define sets
model.Buses = ['Bus1', 'Bus2', 'Bus3']
model.Lines = [('Bus1', 'Bus2'), ('Bus2', 'Bus3')]

# Parameters
model.Pmin = Param(model.Buses, initialize={'Bus1': 0, 'Bus2': 0, 'Bus3': 0})
model.Pmax = Param(model.Buses, initialize={'Bus1': 100, 'Bus2': 100, 'Bus3': 100})
model.Cost = Param(model.Buses, initialize={'Bus1': 20, 'Bus2': 30, 'Bus3': 25})
model.Pd = Param(model.Buses, initialize={'Bus1': 50, 'Bus2': 60, 'Bus3': 70})
model.Pl_max = Param(model.Lines, initialize={('Bus1', 'Bus2'): 50, ('Bus2', 'Bus3'): 50})

# Branch parameters
model.R = Param(model.Lines, initialize={('Bus1', 'Bus2'): 0.1, ('Bus2', 'Bus3'): 0.2})  # Resistance
model.X = Param(model.Lines, initialize={('Bus1', 'Bus2'): 0.2, ('Bus2', 'Bus3'): 0.3})  # Reactance

# Variables
model.Pg = Var(model.Buses, within=NonNegativeReals)
model.Vm = Var(model.Buses, within=NonNegativeReals)
model.Va = Var(model.Buses, within=Reals)

# Objective function: minimize generation cost
def objective_rule(model):
    return sum(model.Cost[bus] * model.Pg[bus] for bus in model.Buses)

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints: power balance and line flow limits
def power_balance_rule(model, bus):
    return model.Pg[bus] - model.Pd[bus] == sum((model.Vm[bus1]**2 * (model.R[(bus1, bus2)]*cos(model.Va[bus1] - model.Va[bus2]) + model.X[(bus1, bus2)]*sin(model.Va[bus1] - model.Va[bus2]))) / (model.Vm[bus1] * model.Vm[bus2]) for (bus1, bus2) in model.Lines if bus2 == bus) - sum((model.Vm[bus2]**2 * (model.R[(bus1, bus2)]*cos(model.Va[bus1] - model.Va[bus2]) + model.X[(bus1, bus2)]*sin(model.Va[bus1] - model.Va[bus2]))) / (model.Vm[bus1] * model.Vm[bus2]) for (bus1, bus2) in model.Lines if bus1 == bus)

model.power_balance_constraint = Constraint(model.Buses, rule=power_balance_rule)

def line_flow_limit_rule(model, bus1, bus2):
    return model.Vm[bus1] * (model.Vm[bus1] * (model.R[(bus1, bus2)] * cos(model.Va[bus1] - model.Va[bus2]) + model.X[(bus1, bus2)] * sin(model.Va[bus1] - model.Va[bus2]))) <= model.Pl_max[(bus1, bus2)]

model.line_flow_limit_constraint = Constraint(model.Lines, rule=line_flow_limit_rule)

# Solve the optimization problem
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)

# Display the optimal power generation and voltage magnitude at each bus
for bus in model.Buses:
    print(f"Optimal power generation at {bus}: {model.Pg[bus].value}")
    print(f"Voltage magnitude at {bus}: {model.Vm[bus].value}")


ValueError: Model constraint (power_balance_constraint[Bus1]) contains nonlinear terms that cannot be written to LP format

In [14]:
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Define sets
model.Buses = ['Bus1', 'Bus2', 'Bus3']
model.Lines = [('Bus1', 'Bus2'), ('Bus2', 'Bus3')]

# Parameters
model.Pmin = Param(model.Buses, initialize={'Bus1': 0, 'Bus2': 0, 'Bus3': 0})
model.Pmax = Param(model.Buses, initialize={'Bus1': 100, 'Bus2': 100, 'Bus3': 100})
model.Cost = Param(model.Buses, initialize={'Bus1': 20, 'Bus2': 30, 'Bus3': 25})
model.Pd = Param(model.Buses, initialize={'Bus1': 50, 'Bus2': 60, 'Bus3': 70})
model.Pl_max = Param(model.Lines, initialize={('Bus1', 'Bus2'): 50, ('Bus2', 'Bus3'): 50})

# Branch parameters
model.R = Param(model.Lines, initialize={('Bus1', 'Bus2'): 0.1, ('Bus2', 'Bus3'): 0.2})  # Resistance
model.X = Param(model.Lines, initialize={('Bus1', 'Bus2'): 0.2, ('Bus2', 'Bus3'): 0.3})  # Reactance

# Variables
model.Pg = Var(model.Buses, within=NonNegativeReals)
model.Vm = Var(model.Buses, within=NonNegativeReals)
model.Va = Var(model.Buses, within=Reals)

# Objective function: minimize generation cost
def objective_rule(model):
    return sum(model.Cost[bus] * model.Pg[bus] for bus in model.Buses)

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints: power balance and line flow limits
def power_balance_rule(model, bus):
    return model.Pg[bus] - model.Pd[bus] == sum((model.Vm[bus1]**2 * (model.R[(bus1, bus2)] + model.X[(bus1, bus2)]) / 2 * (model.Va[bus1] - model.Va[bus2])) for (bus1, bus2) in model.Lines if bus2 == bus) - sum((model.Vm[bus2]**2 * (model.R[(bus1, bus2)] + model.X[(bus1, bus2)]) / 2 * (model.Va[bus1] - model.Va[bus2])) for (bus1, bus2) in model.Lines if bus1 == bus)

model.power_balance_constraint = Constraint(model.Buses, rule=power_balance_rule)

def line_flow_limit_rule(model, bus1, bus2):
    return (model.Vm[bus1] * (model.R[(bus1, bus2)] + model.X[(bus1, bus2)])) <= model.Pl_max[(bus1, bus2)]

model.line_flow_limit_constraint = Constraint(model.Lines, rule=line_flow_limit_rule)

# Solve the optimization problem
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)

# Display the optimal power generation and voltage magnitude at each bus
for bus in model.Buses:
    print(f"Optimal power generation at {bus}: {model.Pg[bus].value}")
    print(f"Voltage magnitude at {bus}: {model.Vm[bus].value}")


ValueError: Model constraint (power_balance_constraint[Bus1]) contains nonlinear terms that cannot be written to LP format

In [8]:
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Define sets
model.Buses = ['Bus1', 'Bus2', 'Bus3']
model.Lines = [('Bus1', 'Bus2'), ('Bus2', 'Bus3'), ('Bus1', 'Bus3')]

# Parameters
model.Pmin = Param(model.Buses, initialize={'Bus1': 0, 'Bus2': 0, 'Bus3': 0})
model.Pmax = Param(model.Buses, initialize={'Bus1': 100, 'Bus2': 100, 'Bus3': 100})
model.Cost = Param(model.Buses, initialize={'Bus1': 20, 'Bus2': 30, 'Bus3': 25})
model.Pd = Param(model.Buses, initialize={'Bus1': 50, 'Bus2': 60, 'Bus3': 70})
model.Pl_max = Param(model.Lines, initialize={('Bus1', 'Bus2'): 50, ('Bus2', 'Bus3'): 50 , ('Bus1', 'Bus3'): 50})
model.Vmin = Param(initialize=0.9)
model.Vmax = Param(initialize=1.1)


# Branch parameters
model.R = Param(model.Lines, initialize={('Bus1', 'Bus2'): 0.1, ('Bus2', 'Bus3'): 0.2 , ('Bus1', 'Bus3'): 0.1})  # Resistance
model.X = Param(model.Lines, initialize={('Bus1', 'Bus2'): 0.2, ('Bus2', 'Bus3'): 0.3 , ('Bus1', 'Bus3'): 0.1})  # Reactance

# Variables
model.Pg = Var(model.Buses, within=NonNegativeReals)
model.Vm = Var(model.Buses, within=NonNegativeReals, bounds=(model.Vmin, model.Vmax))
\
model.Va = Var(model.Buses, within=Reals)

# Objective function: minimize generation cost
def objective_rule(model):
    return sum(model.Cost[bus] * model.Pg[bus] for bus in model.Buses)

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints: power balance and line flow limits
def power_balance_rule(model, bus):
    return model.Pg[bus] - model.Pd[bus] == sum((model.Vm[bus1]**2 * (model.R[(bus1, bus2)]*cos(model.Va[bus1] - model.Va[bus2]) + model.X[(bus1, bus2)]*sin(model.Va[bus1] - model.Va[bus2]))) / (model.Vm[bus1] * model.Vm[bus2]) for (bus1, bus2) in model.Lines if bus2 == bus) - sum((model.Vm[bus2]**2 * (model.R[(bus1, bus2)]*cos(model.Va[bus1] - model.Va[bus2]) + model.X[(bus1, bus2)]*sin(model.Va[bus1] - model.Va[bus2]))) / (model.Vm[bus1] * model.Vm[bus2]) for (bus1, bus2) in model.Lines if bus1 == bus)

model.power_balance_constraint = Constraint(model.Buses, rule=power_balance_rule)

def line_flow_limit_rule(model, bus1, bus2):
    return (model.Vm[bus1] * (model.R[(bus1, bus2)] + model.X[(bus1, bus2)])) <= model.Pl_max[(bus1, bus2)]

model.line_flow_limit_constraint = Constraint(model.Lines, rule=line_flow_limit_rule)

# Solve the optimization problem using ipopt
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print(results)

# Display the optimal power generation and voltage magnitude at each bus
for bus in model.Buses:
    print(f"Optimal power generation at {bus}: {model.Pg[bus].value}")
    print(f"Voltage magnitude at {bus}: {model.Vm[bus].value}")



Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 9
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.14.14\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.033715009689331055
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal power generation at Bus1: 50.250112694874396
Voltage magnitude at Bus1: 1.1000000108142982
Optimal power generation at Bus2: 59.29103184647847
Voltage magnitude at Bus2: 0.8999999900699727
Optimal power generation at Bus3: 70.22448580130371
Voltage magnitude at Bus3: 1.1000000108414323
